<a href="https://colab.research.google.com/github/sajjad-nezhadi/GPT2/blob/main/GPT2_NPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing and importing dependancies

In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.5 MB/s eta 0:00:00


In [ ]:
pip install bertviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.6 MB/s eta 0:00:00


In [ ]:
from transformers import GPT2Tokenizer, GPT2Model, utils
from bertviz import model_view, head_view
import numpy as np
utils.logging.set_verbosity_error()
import random
import csv

#Importing GPT2 model from huggingface




In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', output_attentions=True)
model = GPT2Model.from_pretrained('gpt2', output_attentions=True)

# Helper Functions

In [ ]:
def attentions_get(text):
    encoded_input = tokenizer(text, return_tensors='pt')
    attention = model(**encoded_input).attentions
    tokens = tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][0])
    return attention

def find_head(examples):
    full_attentions = np.zeros((12,12))
    for example in examples:
        text = example[0]
        word = example[1]
        liscensor = example[2]
        att = attentions_get(text)
        for layer in range(12):
            for head in range(12):
                full_attentions[layer][head] += att[layer][0][head][word][liscensor]
    return full_attentions/len(examples)

def illusion_illustration(text, layer=3, head=11):
    heads = [head]
    encoded_input = tokenizer(text, return_tensors='pt')
    attention = model(**encoded_input).attentions
    tokens = tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][0])
    head_view(attention, tokens, layer=layer, heads=heads)

In [ ]:
def attention_weight(filename,layer,head):
  exp = []
  with open(filename, newline='') as csvfile:
      spamreader = csv.reader(csvfile, delimiter=',')
      for row in spamreader:
          insert = [row[0]]
          sent = row[0].split()
          for i in range(len(sent)):
              if (sent[i]=="ever"):
                  insert.append(i)
          for i in range(len(sent)):
              if (sent[i] == "no" or sent[i] == "No" or sent[i] == "not"):
                  insert.append(i)
          exp.append(insert)
  return find_head(exp)[layer,head]

In [ ]:
def return_head(filename, include_rand=False):
  examples = []
  with open(filename, newline='') as csvfile:
      spamreader = csv.reader(csvfile, delimiter=',')
      for row in spamreader:
          sentence = row[0]
          sentence_split = sentence.split()
          for i in range(len(sentence_split)):
              if (sentence_split[i]=="ever"):
                  word = i
                  break
          for i in range(len(sentence_split)):
              if (sentence_split[i] == "no" or sentence_split[i] == "No" or sentence_split[i] == "not"):
                  liscensor = i
                  break
          # Includes random sentences to start in order to avoid head which payes attention to location alone
          if (include_rand):
            rand_sentences = []
            with open('random_sentences.csv', newline='') as csvfile:
                spamreader = csv.reader(csvfile, delimiter=',')
                for row in spamreader:
                    rand_sentences.append((row[0],int(row[1])))
            rand_sentence = random.choice(rand_sentences)
            sentence = rand_sentence[0] + sentence
            word += rand_sentence[1]
            liscensor += rand_sentence[1]

          examples.append((sentence,word,liscensor))
  full_attentions = find_head(examples)
  layer, head = np.unravel_index(full_attentions.argmax(), full_attentions.shape)
  layer = int(layer)
  head = int(head)
  print("layer: ", layer, " head: ", head)
  print("Avg attention weight: ",full_attentions[layer,head])
  indx = np.unravel_index(np.argsort(full_attentions, axis=None), full_attentions.shape)
  vals = full_attentions[indx]
  return layer, head, [(indx[0][i],indx[1][i]) for i in range(len(indx[0]))], vals

## Run function to find best head and the average attention weight. Also returns in ascending order location of heads with attention weights

In [ ]:
layer, head, indx, vals = return_head("gram.csv", include_rand=True)

layer:  3  head:  11
Avg attention weight:  0.37950579325358075


In [ ]:
indx[-5:], vals[-5:]

([(3, 9), (5, 3), (8, 4), (4, 1), (3, 11)],
 array([0.08358218, 0.09188473, 0.09213147, 0.11581545, 0.37950579]))

## Visualize the attention for a sentence

In [ ]:
text = "No bills that no senators voted for will ever become law."
illusion_illustration(text, layer, head)

<IPython.core.display.Javascript object>

## Here we will run three experiments to check avergae weight of attention by picked head on the sentences given.

In [ ]:
# Gramatical with No as first word
weight = attention_weight("gramatical_sentences_exp3.csv",layer,head)
print(weight)

0.2746022542317708


In [ ]:
# UnGramatical with No
weight = attention_weight("ungramatical_sentences_exp3.csv",layer,head)
print(weight)

0.6513264973958334


In [ ]:
# UnGramatical with Not
weight = attention_weight("ungramatical_sentences_exp3_not.csv",layer,head)
print(weight)

0.22067851490444607


## Use under here for new expeiments/finding heads